In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Plot sample

In [ ]:
image, mask, image_input, mask_input = get_sample("1", validation=False, device=DEVICE, go_parent_folder=True)

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image[0], cmap="gray")
ax[1].imshow(mask[0])
plt.show()

In [12]:
import torch
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
sys.path.insert(0, str(Path(os.path.abspath('')).parent))
from utils import setup_data_loaders, get_sample
from dataset.dataset import TeethDataset

In [ ]:
DATA_FOLDER = "data"

train_dataset = TeethDataset(DATA_FOLDER, "training", True)

# print the shape of a random image and its mask
image, mask = train_dataset[1]
print(f"Image shape: {image.shape}")
print(f"Mask shape: {mask.shape}")

Plot an image and a mask side by side:

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image[0], cmap="gray")
ax[1].imshow(mask[0])
plt.show()

Print the dimensions of an image and a mask:

In [ ]:
# Print the dimensions of an image and a mask:
print(f"Image shape: {image.shape}")
print(f"Mask shape: {mask.shape}")